# Randomized Motif Search

In [1]:
from itertools import product
def kmers(text, k):
    patterns = set()
    for i in range(len(text) - k + 1):
        patterns.add(text[i: i+k])
    return patterns

def getProb(km, profile):
    p = 1
    for i, c in enumerate(km):
        p = p*profile[c][i]
    return p

def ProfileMostProbableKmer(text, k, profile):
    kms = kmers(text, k)
    prob = 0
    for km in kms:
        p = getProb(km, profile)
        if p > prob:
            prob = p
            motif = km
    return motif

In [48]:
def score(motifs):
    # Entropy with pseudocounts
    profile = get_profile_laplace(motifs)
    matrix = []
    entropy = 0
    for k, v in profile.items():
        matrix.append(v)
    matrix = np.array(matrix)
    for col in matrix.T:
        col_ent = 0 #- 1/9*np.log(1/9)/np.log(2)
        for p in col:
            col_ent = col_ent - p*np.log(p)/np.log(2)
        entropy = entropy + col_ent
    return entropy

In [49]:
def score(motifs):
    # Entropy no pseudocounts
    motifs_matrix = []
    for motif in motifs:
        row = []
        for c in motif:
            row.append(c)
        motifs_matrix.append(row)
    motifs_matrix = np.array(motifs_matrix)
    length, width = motifs_matrix.shape
    total_entropy = 0
    for i in range(width):
        frecs = Counter(motifs_matrix[:,i])
        entropy = 0
        for k, v in frecs.items():
            p = v/length
            entropy = entropy - p*np.log(p)/np.log(2)
        total_entropy = total_entropy + entropy
    return total_entropy



In [50]:
import numpy as np
from collections import Counter


def get_profile(motifs):
    motifs_matrix = []
    for motif in motifs:
        row = []
        for c in motif:
            row.append(c)
        motifs_matrix.append(row)
    motifs_matrix = np.array(motifs_matrix)
    height, width = motifs_matrix.shape
    profile = {'A': [], 'C': [], 'G': [], 'T': []}
    for i in range(width):
        frecs = dict(Counter(motifs_matrix[:,i]))
        for k in profile.keys():
            if k in frecs:
                profile[k].append(frecs[k]/height)
            else:
                profile[k].append(0)
    return profile


def get_motifs(dna, profile):
    k = len(profile[list(profile.keys())[0]])
    new_motifs = []
    for line in dna:
        new_motifs.append(ProfileMostProbableKmer(line, k, profile))
    return new_motifs
    


In [76]:
def get_profile_laplace(motifs):
    motifs_matrix = []
    klen = len(motifs[0])
    for motif in motifs:
        row = []
        for c in motif:
            row.append(c)
        motifs_matrix.append(row)
    motifs_matrix = np.array(motifs_matrix)
    height, width = motifs_matrix.shape
    profile = {'A': [], 'C': [], 'G': [], 'T': []}
    for i in range(width):
        frecs = dict(Counter(motifs_matrix[:,i]))
        for k in profile.keys():
            if k in frecs:
                profile[k].append((frecs[k] + 1)/(height + 4))
            else:
                profile[k].append(1 / (height + 4))
    return profile

In [88]:
def RandomizedMotifSearchIter(dna, k, t):
    indexes = np.random.randint(0, len(dna[0]) - k, t)
    motifs = []
    for i, idx in enumerate(indexes):
        motifs.append(dna[i][idx: idx + k])
    BestMotifs = motifs
    while True:
        profile = get_profile_laplace(motifs)
        motifs = get_motifs(dna, profile)
        if score(motifs) < score(BestMotifs):
            BestMotifs = motifs
        else:
            return BestMotifs, score(BestMotifs)

In [255]:
dna = """ATGAGGTC
GCCCTAGA
AAATAGAT
TTGTGCTA""".split('\n')
k = 3
t = len(dna)
print(dna)
# Ejercicio
motifs = ['GTC', 'CCC', 'ATA', 'GCT']
profile = get_profile_laplace(motifs)
print(profile)
motifs = get_motifs(dna, profile)
print(' '.join(motifs))

['ATGAGGTC', 'GCCCTAGA', 'AAATAGAT', 'TTGTGCTA']
{'A': [0.25, 0.125, 0.25], 'C': [0.25, 0.375, 0.375], 'G': [0.375, 0.125, 0.125], 'T': [0.125, 0.375, 0.25]}
GTC GCC ATA GCT


['ATGAGGTC', 'GCCCTAGA', 'AAATAGAT', 'TTGTGCTA']

In [89]:
def RandomizedMotifSearch(dna, k, t):
    motifs_arr = []
    scores_arr = []
    for i in range(1000):
        motifs, sc = RandomizedMotifSearchIter(dna, k, t)
        motifs_arr.append(motifs)
        scores_arr.append(sc)
    min_score_index = np.argmin(np.array(scores_arr))
    return motifs_arr[min_score_index]

In [80]:
k = 8 
t = 5
dna = """CGCCCCTCTCGGGGGTGTTCAGTAAACGGCCA
GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG
TAGTACCGAGACCGAAAGAAGTATACAGGCGT
TAGATCAAGTTTCAGGTGCACGTCGGTGAACC
AATCCACCAGCTCCACGTGCAATGTTGGCCTA""".split('\n')


In [95]:
k=15 
t=20
dna = """CCCACCTCCGATCCTTGCTGTTGTCGAGAATTCAATGGTCTTTGTGTTAATGGTAAATTTTTGCGCACCAAATCTTCCCCATGCCACTCAATTCCCGTCTTGACAAGGAACACGGTGCCGGACTGATTATAAACCCTTTAACGTGAAGCTGTTTCTTTCGCAATACCTTCTCGCTCCCCCACCTCCGATCCT
TGCTGTTGTCGAGAATTCAATGGTCTTTGTGTTAATGGTAAATTTTTGCGCACCAAATCTTCCCCATGCCACTCAATTCCCGTCTTGACAAGGAACACGGCGCTACAGTTTGATTTGCCGGACTGATTATAAACCCTTTAACGTGAAGCTGTTTCTTTCGCAATACCTTCTCGCTCCCCCACCTCCGATCCT
ACCCGAGCGATACTTTGCTTTACCGAGTCGTTTTACCGCCGAGACGGCCCGTTCGGATAGTCGCCCTCTCCTTTTCTGCACAGCGCCGTGTGTTGATTCGTCACGTTTCTTGATGGCAGGAGTTCAATAATATCCGTGCCGGGCATAGAGCCAAGGCTGTCGCGTAGTTAGCGGCCTTTGAATCTCGCTGTT
ACGTCAATTTTTGTTTTGGTAAGTTGGATTATTAACGCCACTGGCAGCTATCTAGACAAAAGTTCTTTGACGTTTGTCCTATTGAATGGCTCAACCTAAACCACTGGTTAGCCATACAATTCGCTGTACACGCCGTAGTTTCCGTAGGGATGATGAGTAGGTGGACTTCCCACTACGGGTGCCATTTGTAAA
CGGAACTTGCTTCTATGACTTTCACATCGTGACCGACGGCCAGACAAGTCTTAGATCTGTTAGCGCTCTTACTGATTCCGCGCATAAATACGCTGTCACTCACCACAACCATTGGATCGCCTTTCCGGTCTGTGTAGTTTGATTTTCACACGCATTACTGCAACTCGGGGGTTCGTGAACAACGATATCAGA
AGACCGAGAACAACCTAGACTGTATCCGGAGAAAAGTGATTGACGCCCGCGTTTGATTTCTGAGTACCCTGGGTCAACGCCGACGAGATATCCTATTCCACTTGATCGTCTTGTACTAAGGAATGTAGTCACGACGTAGCTGCTGGATCTGGATCCGCAGTCGGGGCATAAGAGCAGTACGTCGAATCCATT
TGACCAAGGAGTCCTATACGGATGACGCCGTAGTCGCATTCGCACTCTAGAGTAATCTATAGTAATTTCGGGATGTGTCGGGTGTACGCAAAAGTGTATCGAGACGAGAGGCATCTGTGAGTCTGCCAGACCATCAGCTGGCAACGTGCTGTTAATACCTGAGCCCTGGCGATAAGGTGGTCACCCTAACAA
AAATGCGCCGTAGTTTGGGGGTAATGCCCTTATCAATGCTTGCTCAATCACCCCATCCGATGGTGCCCGGTGCCAACCTAACCCGGTAATATCAATACTTGAGTGGTTACTAGTAACGATCTCAATCAGGGCAATGGTGGTGAGGTCAAACTATGCAGTTACCCGTTCTCTGTACAGTCGGACGCCAAAGTA
AGTACTACGTACTGTATCGATTTCTTTCCTCGGCCAGTCACGAAAGCTGGACCCGTGAGCCCCGAGATTAGCCCAGTTGTGCTTAGAATTGGGTTTAGCGAAGTTAAGCTGGCCTGTGTTATTACCGCTAATCTGTCAGACGCATATTACAGGATGTCCGTAGTTTGATCAACATATCGCCTGCACCAAAAC
TCCAGTCACTATCAACCCTAAATGACCACCGGATACTCCGAACAAGTTCATAATAAGGAATACGTGAGTCAGTTGCACGCCTGGTCCTCAGTCATTGGCAGACAGTACGAGTACTGCCAATGTGTTTTCTTTCCCCCCCTAGGCCAGTTCGGTCGGGCTCGTTCTCGCCGTGTCTTGATTTTGCCTTAGTCT
CGATATGAGATAATATCAGATTTGCGGCAGGAGAATATTTTTTAGTCACATATTTATGTACCACAACTCTGATAAGAATGAAGAATGCCGCCGTATAATGATTTGCAGTGTCAACCCAGACAAAGGTTGGCCAAAAAAAGTGTCTGGCTGTACGGGCCTCATCGAGCGCGCGGTCGTAATACAGTACTTCAA
CTCGCACCAGTTTGATTAAAGCTCGCCCAGGATCGCTTGGCATAACTTCATCAACGCCGTGTCTCTATGATTAGTCAAATGGCTATCGCTGTTTCGGCGAACCTATTCCGTTACGCTCATGTCAAACAGCGGCAAGAGATTAGCTGTAGGAGTCTGACTCCCAAATTCGCCGGGATCTTCGAAACATATCAG
CTGGCGGTCGAGTTTCTGACTACGATCTAGTTTGATTATCCGATGTGTAGGTACCACACGGTGCTCGCTGAGTGGCCTAGCCGCCGTGATGAATGGGATCGATAAATGATAGGTACACAAAATCAACATGTACAATGTCAGTGGAGATGGACAAGGCAACGGATTCCGGATGTGCACAAATCTCCTTTCGGA
CGGCTTGGACAGATATCTGGTGCCTGAAGCCGAGACGCATTTCAGGTTTGCACAAGAAGTTTACACGAAGGATGGGCAACTCTGGCGACGACGCCGGCATTTGATTTCAAGATCACCTATCGACTTCTCTAGCGCCGCTGGTTCCCTATAAATGTAGTGAGTTAGAAGTACCCATGTGAAGTAGAATACTTG
ACAAGCCAATTCACAGGAGGACCGATCATTCTCTTCCATCTGGAGATGTCGAGTTCACGAGTCGTCCGCTTATGGGGGGACTCGCGGAGGCTCCAGGTTAGGAGCTGGCGCGCTGCGGTTCCATTCATCCCGTTTAGTGGTTCGTAGTTTGATTGTTGAGAGATCAGTGTATCCGCGCTCAAGGTTTGCCTA
CGATTTCTGATAACCGGTAAGCGACTTGATGAGCAAGGTCGCGCTTGTCCCTCTGTATGCTATAGTTACGATGGCCGCGTTGCAGGCACCAGAGCCCGGCTGGATTTCGGTGGCTGCCTGGCGCTCCTCTCCCGGGTACATCTTTGGCTGTAAGTGGCGCCGTAGTTAACTTCTTAGCTCAGATCCAATTCC
GTTCGCTACAAATCCGTTCAAGCCGCGACGAGCTTCGTTCCAGGTACCACGAACTCGTCGAGCGCCGCTTTTTGATTCAACTACTAGAAGAGGGCACACTGCGCGCAACGCTGTGGGCTGCATGCAAGGCATACTATATGTGTCTCCTGGACGACGGCACAGTTCGGCATCGTCTGCTAGTAGTCTCCCCTC
CTAGCCGGCACAGCTCCGCCTAGGCTAATCTCGGTCGACGGACACGACGCCTCGGTACTCCCGGTTTCACCCGCTGGGTTAGAAGTTAGGGCCGGAAGAGCAGCATGAACACGAACGCCGTAGGGAGATTCAACCGCAACAGCATGACGCTACGTTTCAATGGCTCCGCTGCCTATGATTACATTCTAGTTT
GGTATGTTCGCATTGCTGCGCCTTCACGTAGAAAGGACTCAGGGGCAAGTGGCTAATGCGGCTCGTGGTAATGTCGAGACGAGTCCACTGTGGTGGTGGTCGCGCAGATAGGAGTTAAGTTTGATACGAGAGTGCCGTAGTTTGAGGTGAGAACCAAATGTCGTACCTTGCTGGACCGTAATAGTACTGAGG
TTTTGATCATAGGTACTGTTGCCACTGGATTGGGGATCTATTTAGCGTGGGAAACTGTTGTTACGCTAATCACGTCGAACTACTAACCGCCCGCCTATGTTTGATTCTTCACTCGCGTGGCCCTGGCTCAGTATTATCTATTGTACTGGTAGCAGCCGCGGCGCATTGTATGATGACTGCGTGATCCCGCAT""".split('\n')

In [96]:
best_motifs = RandomizedMotifSearch(dna, k, t)

In [97]:
print('\n'.join(best_motifs))

CTCCGATCCTTGCTG
CGCTACAGTTTGATT
CGCCGTGTGTTGATT
CGCCGTAGTTTCCGT
CTGTGTAGTTTGATT
CGCCCGCGTTTGATT
CGCCGTAGTCGCATT
CGCCGTAGTTTGGGG
GTCCGTAGTTTGATC
CGCCGTGTCTTGATT
CGCCGTATAATGATT
CGCACCAGTTTGATT
CGATCTAGTTTGATT
CGCCGGCATTTGATT
GTTCGTAGTTTGATT
CGCCGTAGTTAACTT
CGCCGCTTTTTGATT
CGCCGTAGGGAGATT
TGCCGTAGTTTGAGG
CGCCTATGTTTGATT


In [ ]:
CATGGGGAAAACTGA
CCTCTCGATCACCGA
CCTATAGATCACCGA
CCGATTGATCACCGA
CCTTGTGCAGACCGA
CCTTGCCTTCACCGA
CCTTGTTGCCACCGA
ACTTGTGATCACCTT
CCTTGTGATCAATTA
CCTTGTGATCTGTGA
CCTTGTGATCACTCC
AACTGTGATCACCGA
CCTTAGTATCACCGA
CCTTGTGAAATCCGA
CCTTGTCGCCACCGA
TGTTGTGATCACCGC
CACCGTGATCACCGA
CCTTGGTTTCACCGA
CCTTTGCATCACCGA
CCTTGTGATTTACGA

In [86]:
score(motifs_arr[min_score_index])

5.8057304727060375

# Gibbs Sampling

In [120]:
np.random.choice(3, p = [0.9, 0.1, 0])

0

In [215]:
def GibbsSamplerIter(dna, k, t, N):
    indexes = np.random.randint(0, len(dna[0]) - k, t)
    motifs = []
    for i, idx in enumerate(indexes):
        motifs.append(dna[i][idx: idx + k])
    BestMotifs = motifs
    for j in range(N):
        i = np.random.randint(t)
        profile = get_profile_laplace(motifs[:i] + motifs[i+1:])
        p = []
        for idx in range(len(dna[i])-k+1):
            p.append(getProb(dna[i][idx:idx+k], profile))
        p = np.array(p)
        p = p/p.sum()
        sel_idx = np.random.choice(len(p), p = p)
        motifs[i] = dna[i][sel_idx:sel_idx+k]
        if score(motifs) < score(BestMotifs):
            BestMotifs = motifs
    return BestMotifs, score(BestMotifs)

In [219]:
def GibbsSampler(dna, k, t, N):
    motifs_arr = []
    scores_arr = []
    for i in range(20):
        motifs, sc = GibbsSamplerIter(dna, k, t, N)
        motifs_arr.append(motifs)
        scores_arr.append(sc)
    min_score_index = np.argmin(np.array(scores_arr))
    return motifs_arr[min_score_index]

In [242]:
k = 15 
t = 20
N = 2000

dna = """TGATGCCCCACAGTGTATGGAGTCCAAAACGTACTGAATTGGATTAACTCGCGGAGTCTAGCCATTCGTATTCTCATTCAACGGAGCAGCGAGATCAGGCCTGTGGAGGGCTTATGTCAGTGCGAATTTCGTTAGTTCGTTCAAGCTATCTTGCTATCCGCCCTCGTTCGGTTTAGATTACAGCTAGCCTGCCGTGCGACGCCGTACGTACAGCGGCGTAGCTTGTGCTTTCAGCAGTGTCATCTGGTGAAGAGCTCCCTGGGAGGTGATGAGTTGAGTCGGCGTGTGGGGCCGGCCCGCGGGCGGTGCACGTACAGTGATGCCCCACAGTG
TATGGAGTCCAAAACGTACTGAATTGGATTAACTCGCGGAGTCTAGCCATTCGTATTCTCATTCAACGGAGCAGCGAGATCAGGCCTGTGGAGGGCTTATGTCAGTGCGAATTTCGTTAGTTCGTTCAAGCTATCTTGCTATCCGCCCTCGTTCGGTTTAGATTACAGCTAGCCTGCCGTGCGACGCCGTACGTACAGCGGCGTAGCTTGTGCTTTCAGCAGTGTACGATAGATTCCTGTCATCTGGTGAAGAGCTCCCTGGGAGGTGATGAGTTGAGTCGGCGTGTGGGGCCGGCCCGCGGGCGGTGCACGTACAGTGATGCCCCACAGTG
GTTTCTGAAGATTGGGGATGCGGCGCGATCCTAGAATGAGAACTGCGTCTTGTGCATGCACTCTGACCGAGGTAACGCATTATACGTGGGAGTGGGAATTTCGCTCGGTGAGGGGCTGGTCTTGGTATACGATTCCTGTGCCCAGGCATGAAAATTCAGAGTGAGTGATAGTTTGACGTTTCGGCTGGAAGCCTACTGTGTCTCCCAGATCCCGTATCTAGTACCGTCATTACTACCGTAGTTATGGTGAAAGAATGCGTAGTCGCCGGAGGATACAAACGATGGTGTGGATGGATCAGGTCTATAGACATAGCCAGCTAAAACTCATTCTA
TGTTTCGTATAGCCCCCAGGCCCATCTTGAACGTACGTACGAGCGCTGTCGTCAAGTCAAAAAGCAACGTGCATGCGTCATCGTCTTGTTAGCTTTGCTCAAACCGTTAATAAGTGCTCGGTCAAGCCTCCCGTTATTAACGAGTATGTGAAAACCTCTGGGTGCCATCGCAAACGTCCACCGCTGTTGCCGCCCCGATGAGCCTAGGGGTAAAATAATCCCGGCTTTAGGCCCTGACTAGCGTTACCCGACTGACCCGGTCGACTCGGCCTCTCTACCTATGGCCCCGGAACACTAAAGTGCGACCAGCAACTTCACGCCTGTGCCAAGTA
AGAGGTTCAACTCCTCCACTGCCAACGGAGGGCATGGAGGTGCAGAAGATATACGCGTTTAGGGCAGATGCGAGCTTTTCGTTACAGTGTATGGTCATAGACGTAAACTTCCTGTGTGTAACTCTGAAAGTCCCATCTAGGCTTCAGTAAACTGCCACTTGCTATCGACCTGCGGCATTGACGATCGGACCTGGGCAGTGGACGCTGCCACTCCTAATATCGGACTCCGATTCGAGCTGATCAAGCGTTTTTCGCACCTCTTGTCATACGCTAGTTTGGGAGGAGGGCATTCCCCGACTAGCTCCGCGGATTTACCCTCTAGACTGGGACCG
CGGTTGCTTTAGTCCGTGTCTTCGACACTTTTAAATCTACTTTTTATCCACCAGAACCGCTACGAGTTGCAAGGGTGGATAAGTATTGTCCATTTCTCACTCTACGTCATTCAACGATCTCTCACCAAGACGTCTGAACGTCGGTGTTGTGGCCCTAAGCTGGTCACTGCCCGGACTCTTGTCTTATTTATACAAACAACGATTCCTGTACTGACATAGTCGAAACCTTTTGACAATGCAGGACTACAACCTCCCGGAGGGATACAATTACTGTCGTGACCCAGGCTCAGGTAATGGATATAATAATGCGACGCTAGTCCATGATGGATGGG
CGCTGACGTGCAAAAAGTAGCTACCGGGTGTCCGTAAGGTAGGGCGACGTGGAGTTGCTCTTCTAACAAAATAGCATCAAAGGAGCGGGCATTGACGAGATCAGTGTTCAGCACCCACATCCCCACACGTGCTTCGGCCCAATTGTAACGTACGGCCCCTGTAGATTGCATTAGAACTCTGAGCTGGATGCCCTCCGACCACGTGCGTTAAAAAGAGCCGTATCAATCATCTAAGACGGATCAAACCAAGCTGTGATAGTAACCGACTTCCACGTGGGATGTAGAGCGTAGTCGCAGATTGCATCTCACTGATCTGACATGGAGGTTTAATT
CACGTTTCCCAGCAAAGATTAAGGCATGGCCGTGCGAATCGGTTCTTGACCTGGCTTAGGGTGCGATGTAACCAGAGCTCCCTATCTCTTGAGCATCTTAAATGACGGTCGGCGGGAGTGCTTTGCTTCCAGACGACCGTTTCACCTGCATGTACCGCTTGTAATAGCTACGTACGATTCGCCTGGTCGCTTACCGGCGCCAATTAGGGTTGTGATAGGGTTGCATAAGGTGACACTCTCCCGCGTGGCGGGGAAACTTGGTTTATCTTCATACATGTAATGTCGACTTGTCCCAGAAGGGGGAGTAATGGCTTAACTGTCCAGCACGATCG
AATGTCCCACGGATGATGTGACCGAGTTGCCTTGTTTACAAATATATAGGATGTTGGACGACGAACGGAGAGAGGAGATGCGGCACCTAATACATTTAGTCAGGTCGCTTGATTCACTACGTGTCGTCTGCAGCTTGAACTGCTCTTCATTATGGGTCCAGGCGTCACATCGCCATAGGTAAGGATTCCGGTCGGAGGTGCCTGAACAGAATGTGGCCACTCGTACTCGCCGACGTACCCATCCTGTATTTAGTCTTGGCTGGTTCAATGAACGCAGTGGTCTAATTCGTCGAATGGAATCACCGAGGGTAATCAAGCGCTCAAGTTACTGG
CGAACGGTTGAACATGCGCACCAATTAGCAGTATCGGCGAGGCTGAAAGCATATAATCAAGATTATCAGCGGTCACCGACAGAGATTATTGATTTAGACAGGAACTGGAGCGTCTCACCCCGTACGATTCCTTGTATAGTTTTTACGCACTCCGCCGCGTATTCAGAATGTCCATCGACCACGAGTCCGATATTTATGTCAGGAAGTAACGGCTCGGCAGAATGATTAACGAATCTCGTGAGCGAAGGCTGCCCCAAATCTGGTCAAACGGGGAGGCCATTTCGGGCCCGGGTTATAACTCTTACTTGGAGTCTTATAAGGGCGTAAGGACC
AACGCCCCTGTACCACTCCTTTCCGTACAAAGCCATCCTGCCTTCAAGCGGGATTTACGTTTAATTCCTGTACTAGCGAACGACTGGGCAGACGCACGTGCGCTTGGGTTGGGTCTTATCTCCTTGACCACGCGGTGCCGTTATTGATAATTCGCGTCACCGGCCGCCATATGAAGCTGTCAATGTGACATTTTGTGTAGAATATCACTATTTACTGTTTGATTCGGCAGCCATACAGATTGGGTAGTAATTAAAAATAGGCGTCTAAAGCAGGGATAGACACTGTGTATTGTAAGTGTTCGATGATCTACCTATAGAACCAGCATAGTTGA
GTGATACACTAATCTTTGACGTATATTTCCTGTCGATGAATAAGCCACTTGCAACCGGGTCACGGCCCGGCAAGCTAGAAATTTATGGACGAAGACCGTAATAATGGATGTGAACTTGGTTGTTGAATCTTATTCCGTTCGGCGAACATGTGCGCGTAGATACGGAACGCAAAGGACTTAGTTTCGCATACGGGAGTAGGACACAAAGTGGCGCTTTTCCAGTGGACGTAATTATGAGTAATCGTAGAATCAAGAAGTGGGTCATTAATCAGCTCCGGAAGGCACGTGATTTAAAGCGCCTCAATCAATCGTTTAGAACTTGCATTTGGATG
GGTCACGTACTGATCCTGTTGCAACGGCATCGCCAAGATAAAAAGTGACACGTACTAGGTGCAGGGGTGCGTCACGTTGCCGAGCTCGCTACGATCTGATCGAGGCCCTTAAGCTTTGCAGCGTTCGCGACTGCCGTGTCCTGGGCCGCTGAACCAGGTGTCCATTCTGATTGTGTCGAAAAGTCCGCGACGCAGGGGCAGCGCGATCGAATGCGACAGGGGTTATCGTGCGGTAAACGTAGTGTGTATTATCGTAAAATAGTCTACGCACCGTCACACCCGGCCTACCCTGAGTTAAGCTCGAGTGGCTCCGCATAGCGAACATACAGCGA
CGTGTTATCCCCCCAATTCGCAGGTAAAAGTACCTAGGTGCTGGTACCTGGCCACAGGCGGTAAAGTATCCAGTGGCGGTTACTACACGTTTCATTCCTGTCTCTATGCGTCAGATAACGCTAATCAATGGGGACGTTTGGGTATGAATTTTAATCCTCGTCCGGTAGCAGTGCATGACGGCGACTTAGCCGGAACCAGTCTTTGAGGGAATGGATCACGCTTACCAGGCGTGTCGGAACTATTCACTACCGGCGAGGGCGGTAAGACTGCTATTTCTTATTCGCCGTGGCCTCTAAGGTTCCTGGTCGCCTTGACGTAGAAAAGTAAGTAG
AAAGTTGTAGCTCCGTTCCTACATCTGGCATTGGAGAATACGCCCTTGCCGGCCTAGTACCCCTTGGTCGCGTCACCGGCAACCCTCTAGTTACGTATGGCCGTGGTTCTGAACGTATGTGATGCTTAGTTTTAGTTTAGGGAAACGACATCGTCAACCGTCATCGCAGCTCACGGGTTTCTTACAGTATACGCCAGATTCTCTAACAGAAACTCTCAGGATTGTACGGCACGTCCGCGCAAAACCGGGGTCCATTCGCCGTCGTGCTTACGTAAGCCTACTAGGGACAGTCACGGTTGCCTGTGTACGTACGATTCCCACTTGCCCAAAGA
AAATTGCTGGTTGGACCTCAGCGTCTCAAGCGCAGCTCGGATGTGCCTCCGCGGCATCATTCGAGTCTTCATTCAGTCACGAATGCCGTATTACCCTGCTAGAGAAAACGCTAGATGTATTCTTAATTGATACTGGGTCTCCGTTGTAACGTCGATCACGACAGATTCCTGTTTGTTGAAATGCGCATACATGCAGTCACAAAGTACACAATGAGGTGATTTGATGGGTTTTCCCCAGATCCGGTTTAGCTCATCTCACAACTGTCGACTTCCGAAACGCGCTGCTATTTTCTCAGGGAAATAAGTATACCTTGCCTTTTACGCCGACATCG
CGTTTAGCAAGGTGAACTTCTGGATTAAGCGAGTTCGGAACGGAGCGGGACATCGCGTTCATTGAAAGCAATGGCGCCAAGGTAAGGCCTTCAATGATAGCGTTCCTGAGATTCCCGCCCAGGTAGACTATTGATCATTTTCAGTAACCAACGCGCTTGTCTTTCTTAGTTTTCGTGTTCGCTTACTCTCCATGGACCGTACCGAGATATAAGTTCCGCGGGCATCTACCAGTTGCTGTATGTCCATTGCCCCACCTACTGTCAGGGATGCCCGCAATCAAGAAGTTTAGCTCGCAGACGTACGATTTTAGTGTAAGGGGCCGGGGACGGAC
GGGTGCAACCATCAGCGTGAGACGATACAAGGCTTCGCCATACAATGGGATCCGAAACACGTACGATAATTGTGTTGTTACGTGTCGATCACTCAATGGGAATCCCACTGCAGGCGCAGTACCGGGGGAAACTTCTCGCTAACATGGTGGGCCCATGCAAGTAGCACAGTGGTGTCTCCATGCGGGGCATTAATTGTTGAAGAGACCCCATTTATCATTCGTTAATGTGTGCCCCCGTTCTGGGACTCGCGATAATAGCTGGAGTAATCGCGGCCACACCTCAAGTGTTTGACACCGGATGCCGGGCGGGCTAGTCCAACCAGATGAGATTT
CCGGGTGTTCTGAGTATTCGTCGCATCTGTAATAGCATTTAAAAGGAAAGCAATGCAACGTTTCACGAGGAGGAGTGTACGATTCCTGGTGTTGGTAGCGGCGGGGAAAATAGGGTCGCAACCATTTTAGAACGACACCGCTCGCAGTCAACCACACACCCCAGCAGCAGGAGGCTTCCCCGTACGTGCATGGACTTCGGGGGACGCTTCCTGCCCGAGACCACCACTTCAGTATGTAGGACTGGACCGGTAAACAACTTACGGGAAGTTATAGTCAACAGTACCACTATTACTCACCTCTAAATAACCGAATCGGAAGCCGATTTATGCGC
TTGATCCTAAGACCATCCACTGATATGTAACGCGCTTTAACTGTACGCGGATTTTTACGGGATAAACCTCTCATTGGGGTCCAGAGTTCAGCTGATGAAGCTATGTGCGTGGCCTCGGTCCCGCATTACGGTGGGCCAAGGCTTACCGTAGCAGTTGGAGGATCGATCTTATCACCGGCAGTGACTAAATGATGGATGGCTGTTACTCCAGATCTGCTGCATTTGGAACAATCTTCTCCCACCGTGAGAGTTTCGTGGGGGCTCCAAGTTGGCGCCTGTCGTGTAAAGCAACTTCGCACGCGTCTCCCGCGAGGTTACCGGCGATTCCTGTG""".split('\n')
%time gibs_found = GibbsSampler(dna, k, t, N)
print(gibs_found)
print(score(gibs_found))

CPU times: user 1min 17s, sys: 725 ms, total: 1min 18s
Wall time: 1min 21s
['CAGTGCGAATTTCGT', 'ACGATAGATTCCTGT', 'GTATACGATTCCTGT', 'ACGTACGAGCGCTGT', 'ACGTAAACTTCCTGT', 'AACAACGATTCCTGT', 'ACGTACGGCCCCTGT', 'ACGTACGATTCGCCT', 'ACGTACCCATCCTGT', 'CCGTACGATTCCTTG', 'ACGTTTAATTCCTGT', 'ACGTATATTTCCTGT', 'ACGTACTGATCCTGT', 'ACGTTTCATTCCTGT', 'ACGTACGATTCCCAC', 'ACGACAGATTCCTGT', 'ACGTACGATTTTAGT', 'ACGTACGATAATTGT', 'GTGTACGATTCCTGG', 'ACCGGCGATTCCTGT']
14.773811532460147


In [241]:
print(score(gibs_found))
print('\n'.join(gibs_found))


15.424917918370056
GGTGAAGAGCTCCCT
ACGATAGATTCCTGT
GTATACGATTCCTGT
ACGTACGAGCGCTGT
ACGTAAACTTCCTGT
AACAACGATTCCTGT
ACGTACGGCCCCTGT
ACGTACGATTCGCCT
ACGTACCCATCCTGT
CCGTACGATTCCTTG
ACGTTTAATTCCTGT
ACGTATATTTCCTGT
ACGTACTGATCCTGT
ACGTTTCATTCCTGT
ACGTACGATTCCCAC
ACGACAGATTCCTGT
ACGTACGATTTTAGT
ACGTACGATAATTGT
GTGTACGATTCCTGG
ACCGGCGATTCCTGT


In [243]:
print(score(gibs_found))
print('\n'.join(gibs_found))

14.773811532460147
CAGTGCGAATTTCGT
ACGATAGATTCCTGT
GTATACGATTCCTGT
ACGTACGAGCGCTGT
ACGTAAACTTCCTGT
AACAACGATTCCTGT
ACGTACGGCCCCTGT
ACGTACGATTCGCCT
ACGTACCCATCCTGT
CCGTACGATTCCTTG
ACGTTTAATTCCTGT
ACGTATATTTCCTGT
ACGTACTGATCCTGT
ACGTTTCATTCCTGT
ACGTACGATTCCCAC
ACGACAGATTCCTGT
ACGTACGATTTTAGT
ACGTACGATAATTGT
GTGTACGATTCCTGG
ACCGGCGATTCCTGT


In [237]:
score([
    'ACGTCCACCGGCGTC',
    'AAGCGCACCGGGGTG',
    'ACCCTTACCGGGGTG',
    'AAGTTCCTCGGGGTG',
    'AAGTTTTATGGGGTG',
    'AAGTTTACCGGGTGC',
    'AAGTTTCGAGGGGTG',
    'CTGTTTACCGGGGTA',
    'AAGTTGCTCGGGGTG',
    'AAACATACCGGGGTG',
    'AAGTTTAGGAGGGTG',
    'AAGGAAACCGGGGTG',
    'AAGTTTACACAGGTG',
    'TAGTTTACCGGGGAT',
    'CCTTTTACCGGGGTG',
    'AAGTGAGCCGGGGTG',
    'AAGTCGTCCGGGGTG',
    'AAGTTTACCGGACAG',
    'AAGTTTACCAATGTG',
    'AAGTTTACCGTCATG'
])

14.866132778122266

In [226]:
score(['TCTCGGGG',
'CCAAGGTG',
'TACAGGCG',
'TTCAGGTG',
'TCCACGTG'])

6.278636068026094

In [126]:
motifs = ['GGGTGTTC', 'GAGGTATG', 'AGACCGAA', 'TTTCAGGT', 'CGTGCAAT']

In [130]:
de = 4
motifs[:de] + motifs[de+1:]

['GGGTGTTC', 'GAGGTATG', 'AGACCGAA', 'TTTCAGGT']

In [247]:
k = 15
p1 = (600- k) / (600-k+1)
print(p1)
pC = pow(p1,10)
print(pC)
pAnswer = 1 - pC
print(pAnswer)

0.9982935153583617
0.9830656030708897
0.01693439692911025
